# Embeddings

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [1]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/maryszmary/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7f6b8d323630>)

In [3]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [4]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [7]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297019481658936
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.5553582906723022
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681 

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Находим косинусную близость пары слов:

In [8]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [9]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [10]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

/usr/local/lib/python3.5/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'ананас_S'

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [11]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2019-09-10 17:10:46--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: ‘unlabeledTrainData.tsv’

unlabeledTrainData. 100%[===================>]  64,16M  2,47MB/s    in 43s     

2019-09-10 17:11:32 (1,48 MB/s) - ‘unlabeledTrainData.tsv’ saved [67281491/67281491]



In [12]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [13]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [14]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [15]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


/usr/lib/python3/dist-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/usr/lib/python3/dist-packages/bs4/__init__.py:198: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


In [16]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [17]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [18]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 4min 20s, sys: 910 ms, total: 4min 21s
Wall time: 1min 18s


Смотрим, сколько в модели слов.

In [19]:
print(len(model_en.wv.vocab))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [20]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7609376311302185)]
[('men', 0.633162260055542)]
[('germany', 0.7396211624145508), ('europe', 0.7237502336502075), ('canada', 0.7159187197685242)]
novel


/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.5/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [27]:
model_en.wv.similarity('lion', 'rabbit')

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.32584286

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [29]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2019-09-10 19:06:46--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.244.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: ‘alice.txt’

alice.txt           100%[===================>] 163,70K  --.-KB/s    in 0,1s    

2019-09-10 19:06:47 (1,18 MB/s) - ‘alice.txt’ saved [167631/167631]



In [30]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [31]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [36]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(97048, 150225)

Лев и кролик стали ближе друг к другу!

In [37]:
model.wv.similarity('lion', 'rabbit')

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.3271467

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [38]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [21]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2019-09-10 18:58:21--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.244.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871776 (851K) [text/plain]
Saving to: ‘ru_analogy_tagged.txt’

ru_analogy_tagged.t 100%[===================>] 851,34K  1,55MB/s    in 0,5s    

2019-09-10 18:58:22 (1,55 MB/s) - ‘ru_analogy_tagged.txt’ saved [871776/871776]



In [39]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.evaluate_word_analogies() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [40]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [47]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model[top_words]


['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'they', 'at', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'or', 'her', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can']


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [49]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 17.3 s, sys: 1.15 s, total: 18.5 s
Wall time: 18.5 s


In [ ]:
!pip install bokeh

In [54]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD, и потом применять t-SNE:

In [56]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [57]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [3]:
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

fatal: destination path 'fastText' already exists and is not an empty directory.
Processing ./fastText
  Using cached https://files.pythonhosted.org/packages/5d/85/c7a8dffda52ce25a8bcfe9a28b6861bdd52da59ae001fdd4173e054b7d9b/pybind11-2.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b2/86/095d2f7829badc207c893dd4ac767e871f6cd547145df797ea26baea4e2e/setuptools-41.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/21/2b18339d24a2f73dcefb2f10f48aff6182e16da83e3a612684443c6cfb29/numpy-1.17.2-cp35-cp35m-manylinux1_x86_64.whl
  Running setup.py install for fasttext ... done
You are using pip version 8.1.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [2]:
ft_model.get_word_vector("movie")

array([-6.73057232e-03,  2.08648682e-01, -1.88893840e-01, -4.85788099e-02,
        6.45734444e-02,  1.29233301e-02, -2.47105390e-01, -6.73017651e-03,
       -8.79168790e-03,  8.76431391e-02,  1.43808842e-01,  8.35265443e-02,
       -1.74307749e-02,  3.79030369e-02,  3.74617018e-02, -9.29040164e-02,
       -5.69233031e-04, -9.27814022e-02,  1.15031146e-01,  1.22447662e-01,
        5.78474961e-02, -3.64004560e-02,  9.89502743e-02, -7.79608786e-02,
        9.53299105e-02,  2.64451802e-02,  1.28836200e-01,  9.55208484e-03,
       -9.69734341e-02, -1.23743024e-02,  7.17910826e-02,  3.13416235e-02,
       -2.42757481e-02, -3.46199162e-02, -1.83102563e-02, -2.79114731e-02,
        9.18026268e-02, -3.58579867e-02, -7.19081089e-02, -1.66802078e-01,
        7.83748850e-02,  9.05971229e-02, -9.21034664e-02, -3.60868834e-02,
        9.05119032e-02, -5.10555767e-02,  7.91546479e-02,  1.12251854e-02,
       -2.90307347e-02, -5.20794950e-02,  1.09920003e-01, -4.71951626e-02,
        6.38564071e-03, -

In [3]:
ft_model.get_nearest_neighbors('actor')

[(0.999921977519989, 'actors'),
 (0.9998977780342102, 'director'),
 (0.9998862743377686, 'directors'),
 (0.9998846650123596, 'perfectly'),
 (0.9998726844787598, 'predator'),
 (0.9998723268508911, 'utterly'),
 (0.9998608827590942, 'dialog'),
 (0.9998583793640137, 'actual'),
 (0.9998502135276794, 'attractive'),
 (0.9998490810394287, 'dialogue')]

In [4]:
ft_model.get_analogies("woman", "man", "actor")

[(0.9998149275779724, 'act'),
 (0.9998092651367188, 'badly'),
 (0.9998079538345337, 'actually'),
 (0.9997990131378174, 'poorly'),
 (0.999788761138916, 'fly'),
 (0.999783992767334, 'write'),
 (0.9997831583023071, 'absolutely'),
 (0.9997802972793579, 'simply'),
 (0.9997757077217102, 'silly'),
 (0.9997753500938416, 'enjoyable')]

In [6]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.9999391436576843, 'act'),
 (0.9998903274536133, 'actors'),
 (0.9998863339424133, 'actor'),
 (0.9998792409896851, 'actress'),
 (0.9998623728752136, 'single'),
 (0.9998517632484436, 'actual'),
 (0.9998226761817932, 'terrible'),
 (0.9998196363449097, 'exact'),
 (0.9998190402984619, 'plot'),
 (0.9998172521591187, 'wrong')]

In [7]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.9999324679374695, 'reviews'),
 (0.9999246597290039, 'review'),
 (0.9999151825904846, 'recommended'),
 (0.9999132752418518, 'rented'),
 (0.9998916387557983, 'waste'),
 (0.999889075756073, 'movie'),
 (0.9998835921287537, 'thank'),
 (0.9998812079429626, 'not'),
 (0.9998751878738403, 'watchable'),
 (0.9998645782470703, 'only')]

Кроме этого, fastText можно использовать для классификации, для этого нужен следующий формат размеченных данных:

__label_1__  text_1

__label_2__  text_2

...

In [7]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2019-09-08 11:18:01--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2019-09-08 11:18:02--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdc27882d59b7fbb4ee5d53213f.dl.dropboxusercontent.com/cd/0/inline/AoKSln6uG-7jh4tWkOpriQZeCuEWcU3dtZD2B0Tl6ZmHPmemBlq3X3Cd8QHqM03LlTapMG_xDeAdozZnk-4zhZzHCQRw9SoIDyNO-8oqtxXNQw/file# [following]
--2019-09-08 11:18:02--  https://ucdc27882d59b7fbb4ee5d53213f.dl.dropboxusercontent.com/cd/0/inline/AoKSln6uG-7jh4tWkOpriQZeCuEWcU3dtZD2B0Tl6ZmHPmemBlq3X3Cd8QHqM03LlTapMG_xDeAdozZnk-4zhZzHCQRw9SoIDyNO-8oqtxXNQw/file
Resolving ucdc

In [8]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2019-09-08 11:18:05--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2019-09-08 11:18:06--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb96cff5d4b82d5d36bc949e0a2.dl.dropboxusercontent.com/cd/0/inline/AoJVp2lAGycoFn43QrVjalbNX5WsQ2E_SEQsbfeb5NaBQgPxBB2wYp4G5GBL8v16jXq_YspfbwclseHSwwq6VmVp9GUnhUnHYJmmsS-CHwD8-g/file# [following]
--2019-09-08 11:18:06--  https://ucb96cff5d4b82d5d36bc949e0a2.dl.dropboxusercontent.com/cd/0/inline/AoJVp2lAGycoFn43QrVjalbNX5WsQ2E_SEQsbfeb5NaBQgPxBB2wYp4G5GBL8v16jXq_YspfbwclseHSwwq6VmVp9GUnhUnHYJmmsS-CHwD8-g/file
Resolving ucb9

In [12]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [13]:
len(df)

226834

Проведем стандартный препроцессинг:

In [2]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 3.4MB/s 


In [14]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

m = pymorphy2.MorphAnalyzer()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [15]:
#@lru_cache(maxsize=128)
# если вы работаете не колабе, можно заменить pymorphy на mystem и раскомментирвать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "    

In [16]:
def clean_text(text):
    return lemmatize(words_only(text))

In [18]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))

    
df['lemmas'] = lemmas
df.head()

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


Запишем полученные данные в формате для обучения классификатора:

In [22]:
X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [20]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [21]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8975499625948488
R@1: 0.8975499625948488
Number of examples: 74856


## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [23]:
! wget -O lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

--2019-09-11 00:07:02--  https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ja23c9l1ppo9ix7/lenta-ru-news-part.csv [following]
--2019-09-11 00:07:03--  https://www.dropbox.com/s/raw/ja23c9l1ppo9ix7/lenta-ru-news-part.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc50cd882bc6655beeb161423a6a.dl.dropboxusercontent.com/cd/0/inline/AoXU7mDPrPWV2skQGLHPZu_jL0__EnRHkmdplTBKcEnWm_o-aIpFOGmuI5sP1X0-sVtwQi9YYxtNRQxZetaqJjYC1nYEFuxdo1BYYoedVSvz_pG9FyINvLsde_6c1XAvTKI/file# [following]
--2019-09-11 00:07:04--  https://uc50cd882bc6655beeb161423a6a.dl.dropboxusercontent.com/cd/0/inline/AoXU7mDPrPWV2skQGLHPZu_jL0__EnRHkmdplTBKcEnWm_o-aIpFOGmuI5sP1X0-sVtwQi9YYxtN

In [3]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()

,title,text,topic
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
1,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт
2,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика
3,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика
4,Хоккеист НХЛ забросил шайбу с отрицательного угла,Нападающий клуба «Эдмонтон Ойлерс» Коннор Макд...,Спорт


In [4]:
len(lenta)

258297

In [5]:
lenta.topic.value_counts()

Экономика          79538
Спорт              64421
Культура           53803
Наука и техника    53136
Бизнес              7399
Name: topic, dtype: int64